In [1]:
import numpy as np
import scipy.io as io
import h5py
import deepdish as dd
import dask.array as da

### Read mat and write out as h5

In [4]:
# Get the file pointer
f = h5py.File('sessiondata.mat')
f.keys()

[u'#refs#', u'eyes', u'spikes']

In [5]:
# Eyes data
eyes = np.array(f['eyes'])
eyes

array([[ 0.14579592,  0.4386571 ,  0.45526052, ...,  0.97087662,
         0.77191538,  0.60585405],
       [ 0.84238319,  0.67859425,  0.9235082 , ...,  0.31956551,
         0.39432979,  0.8385655 ],
       [ 0.16025677,  0.23317229,  0.1638342 , ...,  0.8099979 ,
         0.39543674,  0.37210186],
       ..., 
       [ 0.21062345,  0.4310531 ,  0.43279427, ...,  0.22739123,
         0.91842785,  0.46795266],
       [ 0.56451693,  0.63680506,  0.68741876, ...,  0.85187091,
         0.99911962,  0.56117048],
       [ 0.37230056,  0.99421934,  0.63892119, ...,  0.47302873,
         0.37639028,  0.42562775]])

In [6]:
# Convert spikes struct to dict
spikes_object = f['spikes']
unit_object = spikes_object['unit']
times_object = spikes_object['times']

spikes = {}
for u in range(unit_object.size):
    uu = np.array(f[unit_object[u,0]])
    tt = np.array(f[times_object[u,0]])
    channel = int(np.floor(uu[0][0]))
    unit = int(10*(uu[0][0] - np.floor(uu[0][0])))
    unitid = '{0:0=2d}'.format(channel)+'{0:0=2d}'.format(unit)
    times = tt[0][:]
    spikes[unitid] = times

for u in spikes:
    print u
    print spikes[u][:10]

0101
[ 0.10838973  0.28835025  0.41181791  0.45435021  0.45838425  0.47751364
  0.50879078  0.53368318  0.53619985  0.60082462]
0201
[ 0.00493942  0.09943164  0.1250208   0.16063269  0.31444801  0.32342396
  0.45609411  0.48181167  0.57149892  0.65069226]


In [8]:
# Put everything into a bigger dict
d = {}
d['eyes'] = eyes
d['spikes'] = spikes

# Save it using deepdish
dd.io.save('testfile.h5', d)

zlib


In [9]:
# Read it in using deepdish
dat = dd.io.load('testfile.h5')

In [10]:
dat['spikes']['0101'][:10]

array([ 0.10838973,  0.28835025,  0.41181791,  0.45435021,  0.45838425,
        0.47751364,  0.50879078,  0.53368318,  0.53619985,  0.60082462])

In [11]:
# Read in only a small portion of the file with dask

fr = h5py.File('testfile.h5')
unitid = '0101'
x = da.from_array(fr['/spikes/{}'.format(unitid)], chunks=(100, 1))

In [12]:
x.shape

(2000,)